In [2]:
from pymoo.util.nds.non_dominated_sorting import NonDominatedSorting
from generic_optimizer import GenericOptimizer
from joblib import Parallel, delayed
import pandas as pd
import numpy as np
import pickle
import os

from scipy.stats import ranksums

import re

from novel_sampling import over_sample, BiasedBinarySampling
from generic_optimizer import GenericOptimizer, prepare_splits

import matplotlib.pyplot as plt

In [3]:
execution_configs = []
path_string = "optimizations"
for file in os.listdir(path_string):
    with open(os.path.join(path_string, file), 'rb') as fh:
        execution_configs.append(pickle.load(fh))

In [37]:
def calculate(execution_config):

    # print(f'{execution_config["Result"].F.shape} <> {execution_config["Run Name"]}')

    if os.path.exists(f"calculations/{execution_config['Run Name']}.pickle"): return

    x_train = execution_config["x"][execution_config["train_idx"]]
    y_train = execution_config["y"][execution_config["train_idx"]]
    if execution_config["Over Sample"] == "over_sample": 
        x_train, y_train = over_sample(x_train, y_train)
    
    x_validation = execution_config["x"][execution_config["validation_idx"]]
    y_validation = execution_config["y"][execution_config["validation_idx"]]
    x_test = execution_config["x"][execution_config["test_idx"]]
    y_test = execution_config["y"][execution_config["test_idx"]]
    
    validation_fitness = execution_config["Result"].F
    optimization_population = execution_config["Result"].X
    
    if len(optimization_population.shape) == 1:
        optimization_population = np.array([list(optimization_population)])
    
    validation_pareto_front, best_x_train, best_y_train = GenericOptimizer.calculate_optimal_instance(
        x_train,
        y_train,
        x_validation,
        y_validation,
        validation_fitness,
        optimization_population,
        GenericOptimizer.n_neighbours
    )
    execution_config["Best X"] = best_x_train
    execution_config["Best Y"] = best_y_train
    reduction_rate = 1 - (best_x_train.shape[0] / x_train.shape[0])
    
    optimized_ir = GenericOptimizer.calculate_IR(best_y_train)

    optimized_test_err = GenericOptimizer.calculate_overall_error(
        best_x_train,
        best_y_train,
        x_test,
        y_test,
        GenericOptimizer.n_neighbours
    )
    optimized_validation_err = GenericOptimizer.calculate_overall_error(
        best_x_train,
        best_y_train,
        x_validation,
        y_validation,
        GenericOptimizer.n_neighbours
    )
    baseline_test_err = GenericOptimizer.calculate_overall_error(
        x_train,
        y_train,
        x_test,
        y_test,
        GenericOptimizer.n_neighbours
    )
    baseline_validation_err = GenericOptimizer.calculate_overall_error(
        x_train,
        y_train,
        x_validation,
        y_validation,
        GenericOptimizer.n_neighbours
    )

    test_fitness = []
    for objective in execution_config['Executable Objectives']:
        test_fitness.append(GenericOptimizer.unbound_eval_objectives(
            objective,
            optimization_population,
            x_train, y_train,
            x_test, y_test
        ))

    test_fitness = np.column_stack(test_fitness)
    
    test_pareto_front, best_x_FOR_TEST, best_y_FOR_TEST = GenericOptimizer.calculate_optimal_instance(
        x_train,
        y_train,
        x_test,
        y_test,
        test_fitness,
        optimization_population,
        GenericOptimizer.n_neighbours
    )

    ideal_optimized_test_accuracy = GenericOptimizer.calculate_overall_error(
        best_x_FOR_TEST,
        best_y_FOR_TEST,
        x_test,
        y_test,
        GenericOptimizer.n_neighbours
    )

    if False:
        rank_one_pareto_front_metrics = test_fitness[test_pareto_front]
        x = rank_one_pareto_front_metrics[:,0]
        y = rank_one_pareto_front_metrics[:,1]
        x1 = validation_fitness[validation_pareto_front][:,0]
        y1 = validation_fitness[validation_pareto_front][:,1]
        plt.scatter(x, y)
        plt.scatter(x1, y1)
        plt.xlim([0, 1])
        plt.show()
        print(metrics)
        print(fronts)

    execution_config["Optimized Test Error"] = optimized_test_err
    execution_config["Optimized Validation Error"] = optimized_validation_err
    execution_config["Optimized Test Error (Ideal)"] = ideal_optimized_test_accuracy
    execution_config["Optimized Imbalance Ratio"] = optimized_ir
    execution_config["Reduction Rate"] = reduction_rate
    execution_config["Baseline Test Error"] = baseline_test_err
    execution_config["Baseline Validation Error"] = baseline_validation_err
    execution_config["Run Type"] = "_".join(execution_config["Run Name"].split("_")[1:])

    with open(f"calculations/{execution_config['Run Name']}.pickle", 'wb') as fh:
        pickle.dump(execution_config, fh)
        
# for execution_config in execution_configs:
#     calculate(execution_config)

done = Parallel(n_jobs=-1)(delayed(calculate)(execution_config) for execution_config in execution_configs)
print(f"Completed {len(done)} calculations")

Completed 4340 calculations


In [38]:
calculations = []
path_string = "calculations"
for file in os.listdir(path_string):
    with open(os.path.join(path_string, file), 'rb') as fh:
        calculations.append(pickle.load(fh))

In [39]:
aggregated_results = {}
for config in calculations:
    config["Run Type"] = "_".join(config["Run Name"].split("_")[1:])

    pattern = r'\.([a-zA-Z_][a-zA-Z0-9_]*)\s'
    matches = [re.search(pattern, str(name)).group(1) for name in config['Executable Objectives']]
    
    if config["Run Type"] not in aggregated_results:
        aggregated_results[config["Run Type"]] = []

    aggregated_results[config["Run Type"]].append([
        config['Optimized Test Error'],
        config['Optimized Validation Error'],
        config['Optimized Test Error (Ideal)'],
        config['Baseline Test Error'],
        config['Baseline Validation Error'],
        config['Over Sample'],
        config['Initial Population'],
        config['Dataset'],
        "+".join(matches)
    ]) 

In [40]:
def summary_stats(ls):
    my_dict = {
        "Median": np.median(ls),
        "Min": np.min(ls),
        "Mean": np.mean(ls),
        "Max": np.max(ls),
        "Std Dev": np.std(ls),
        "Variance": np.var(ls)
    }
    return my_dict

validation_statistics = []
test_statistics = []
ideal_test_statistics = []
for key in aggregated_results:
    
    reduction_rates, imbalance_ratios = [], []
    optimized_test_error, optimized_validation_error, optimized_posteri_test_error, baseline_validation_error, baseline_test_error = [], [], [], [], []
    sample_option, pop_option, optimzizer, dataset = None, None, None, None

    for ote, ove, oteP, bte, bve, sample, pop, dataset, optimizer in aggregated_results[key]:
        optimized_test_error.append(ote)
        optimized_validation_error.append(ove)
        optimized_posteri_test_error.append(oteP)
        baseline_validation_error.append(bve)
        baseline_test_error.append(bte)

        sample_option = sample
        pop_option = pop
        dataset = dataset
        optimizer = optimizer

    base_line_test_acc = 1 - np.mean(baseline_test_error)
    base_line_validation_acc = 1 - np.mean(baseline_validation_error)

    optimized_test_acc = 1 - np.mean(optimized_test_error)
    optimized_ideal_test_acc = 1 - np.mean(optimized_posteri_test_error)
    optimized_validation_acc = 1 - np.mean(optimized_validation_error)

    optimization_err_diff_test = np.subtract(baseline_test_error, optimized_test_error)
    optimization_acc_diff_test = np.mean(optimization_err_diff_test)
    optimization_err_diff_ideal_test = np.subtract(baseline_test_error, optimized_posteri_test_error)
    optimization_err_diff_ideal_test = np.mean(optimization_err_diff_ideal_test)

    optimization_err_diff_validation = np.subtract(baseline_validation_error, optimized_validation_error)
    optimization_acc_diff_validation = np.mean(optimization_err_diff_validation)
    
    # print(ranksums(baseline_validation_error, optimized_validation_error).__dir__())
    temp = {"DB": dataset, "Sampling": sample_option, "Population": pop_option, "Optimizers": optimizer}
    temp.update(summary_stats(np.subtract(1, optimized_validation_error)))
    temp["Mean Baseline Acc"] = 1 - np.mean(baseline_validation_error)
    temp["Baseline diff"] = temp["Mean"] - temp["Mean Baseline Acc"]
    temp["Pval"] = ranksums(baseline_validation_error, optimized_validation_error).pvalue
    validation_statistics.append(temp)


    temp = {"DB": dataset, "Sampling": sample_option, "Population": pop_option, "Optimizers": optimizer}
    temp.update(summary_stats(np.subtract(1, optimized_test_error)))
    temp["Mean Baseline Acc"] = 1 - np.mean(baseline_test_error)
    temp["Baseline diff"] = temp["Mean"] - temp["Mean Baseline Acc"]
    temp["Pval"] = ranksums(baseline_test_error, optimized_test_error).pvalue
    test_statistics.append(temp)

    temp = {"DB": dataset, "Sampling": sample_option, "Population": pop_option, "Optimizers": optimizer}
    temp.update(summary_stats(np.subtract(1, optimized_posteri_test_error)))
    temp["Mean Baseline Acc"] = 1 - np.mean(baseline_test_error)
    temp["Baseline diff"] = temp["Mean"] - temp["Mean Baseline Acc"]
    temp["Pval"] = ranksums(baseline_test_error, optimized_posteri_test_error).pvalue
    ideal_test_statistics.append(temp)

print(validation_statistics)
pd.DataFrame.from_records(validation_statistics).to_excel("validation.xlsx", index=False)
pd.DataFrame.from_records(test_statistics).to_excel("test.xlsx", index=False)
pd.DataFrame.from_records(ideal_test_statistics).to_excel("ideal_test.xlsx", index=False)

[{'DB': 'australian', 'Sampling': 'over_sample', 'Population': 'bias_population', 'Optimizers': 'calculate_class0_error+calculate_class1_error', 'Median': np.float64(0.6511627906976745), 'Min': np.float64(0.6104651162790697), 'Mean': np.float64(0.6534133533383345), 'Max': np.float64(0.7209302325581395), 'Std Dev': np.float64(0.027056197962236312), 'Variance': np.float64(0.0007320378481717203), 'Mean Baseline Acc': np.float64(0.6537884471117779), 'Baseline diff': np.float64(-0.0003750937734433579), 'Pval': np.float64(0.5878012271190365)}, {'DB': 'australian', 'Sampling': 'over_sample', 'Population': 'bias_population', 'Optimizers': 'calculate_overall_error+calculate_num_examples', 'Median': np.float64(0.8313953488372093), 'Min': np.float64(0.7848837209302325), 'Mean': np.float64(0.8268942235558889), 'Max': np.float64(0.877906976744186), 'Std Dev': np.float64(0.02163871733211698), 'Variance': np.float64(0.0004682340877792598), 'Mean Baseline Acc': np.float64(0.6474118529632409), 'Baselin

In [4]:
execution_types = {}
for result_name, test_err in output:
    if result_name not in execution_types:
        execution_types[result_name] = []
    execution_types[result_name].append(test_err)

len(execution_types["over_sample_rand_population_australian_calculate_overall_error_calculate_num_examples"])


NameError: name 'output' is not defined

In [18]:
for conf in redo:
    print(conf["Run Name"])

0_regular_sample_bias_population_magic_calculate_class0_error_calculate_class1_error
0_regular_sample_bias_population_phoneme_calculate_class0_error_calculate_class1_error
0_regular_sample_bias_population_pima_calculate_class0_error_calculate_class1_error
0_regular_sample_bias_population_segment0_calculate_class0_error_calculate_class1_error
0_regular_sample_bias_population_sonar_calculate_class0_error_calculate_class1_error
0_regular_sample_bias_population_vehicle0_calculate_class0_error_calculate_class1_error
0_regular_sample_bias_population_yeast1_calculate_class0_error_calculate_class1_error
10_regular_sample_bias_population_magic_calculate_class0_error_calculate_class1_error
10_regular_sample_bias_population_phoneme_calculate_class0_error_calculate_class1_error
10_regular_sample_bias_population_pima_calculate_class0_error_calculate_class1_error
10_regular_sample_bias_population_segment0_calculate_class0_error_calculate_class1_error
10_regular_sample_bias_population_sonar_calculate

In [ ]:
p